# Exercise 6. ICT Project: Communication Services and Security
### Cèsar Fernàndez Camón

#### Authors:
- Albert Pérez Datsira
- Jeongyun Lee


## Problem 1
Let's consider a WEP (Wireless Encryption Protocol) cipher consisting on; 8 bits key length, 8 bits IV length, 8 bits CRC (Cyclic Redundancy Check) length being (x^8 + 1) the CRC polynomial.

In addition, the PRNG (Pseudorandom Number Generator) is implemented as 8 bits shift register,

denoting:
* PS(i) as the shift register status at iteration i
* PS(i)[j] as the j th bit of the shift register at iteration i. Consider P S(i)[0] as the most left bit
* PO(i) as the output bit of the PNRG at iteration i

being:
- PS(0) = IV ⊕ key
- PS(i)[0] = PS(i - 1)[4] ⊕ PS(i - 1)[7]
- PS(i)[j] = PS(i - 1)[j - 1], j > 0
- PO(i) = PS(i)[4] ⊕ PS(i)[7]

where (⊕) means a XOR operation.

1. Probe that the ciphered data for the clear data 0x0123, key=0x33 and IV=0x11 is: **0x667E92**
2. Probe how deciphering with a wrong key (key=0x22) an error condition is reported

## Utils
First of all, will be introduced the fucntions amount provided used throughout the code, to compute the solution.
### Calculate CRC (Cyclic Redundancy Code)
Focused on generating the CRC for an inputted message and a divisor. Some comments were added specifying at each step its behavior.

In [2]:
def crc(msg, div, code='00000000'):
    """Cyclic Redundancy Check
    Generates an error detecting code based on an inputted message
    and divisor in the form of a polynomial representation.
    Arguments:
        msg: The input message of which to generate the output code.
        div: The divisor in polynomial form. For example, if the polynomial
            of x^3 + x + 1 is given, this should be represented as '1011' in
            the div argument.
        code: This is an option argument where a previously generated code may
            be passed in. This can be used to check validity. If the inputted
            code produces an outputted code of all zeros, then the message has
            no errors.
    Returns:
        An error-detecting code generated by the message and the given divisor.
    """
    # Append the code to the message. If no code is given, default to '000'
    msg = msg + code

    # Convert msg and div into list form for easier handling
    msg = list(msg)
    div = list(div)

    # Loop over every message bit (minus the appended code)
    for i in range(len(msg)-len(code)):
        # If that messsage bit is 1, perform modulo 2 multiplication
        if msg[i] == '1':
            for j in range(len(div)):
                # Perform modulo 2 multiplication on each index of the divisor
                msg[i+j] = str((int(msg[i+j])+int(div[j]))%2)

    # Output the last error-checking code portion of the message generated
    return ''.join(msg[-len(code):])

### Print as binary

In [3]:
def printAsBinary(val):
  for i in val:
    print('{0:08b}'.format(i))
    
def printAsBinaryString(val):
  temp = ''
  for i in val:
    temp += '{0:04b}'.format(i)
  return temp

def hexToBin(hex,n=24):
  return f'{hex:0>{n}b}'

In [4]:
def listToString(string):   
    temp = ""  
    for s in string:  
        temp += s    
    return temp

### PRNG (Pseudorandom Number Generator)
Next, stands the function regarding the PRNG following the statement problem constraints and using the notations presented.
>Note: The PNRG is implemented as a 8 bits shift register

In [13]:
def xor(a, b): ## simply performing XOR operation among two input values
    return a ^ b

In [15]:
## from the initialization vector and the key there are PS & PO both calculated
def pnrg(iv, key):
  PS = ['{0:08b}'.format(xor(iv, key))]
  PO = []
  
  # iterating over PS once for each bit in ICV + Data
  for i in range(1, len(ICVData)):
    a = int(PS[i-1][4])
    b = int(PS[i-1][7])
    temp = str(xor(a, b))
    PO.append(temp)

    # getting the other bits
    for j in range(1,8):
      temp += PS[i-1][j-1]
    PS.append(temp)

  # last PO value
  PO.append(PS[-1][0])
  
  return PS, PO;

### Data cyphering
- XOR (⊕) between PO and the Data + ICV

In [6]:
def cypher(PO, ICVData):
  cyphered = ''
  for i in range(0, len(ICVData)):
    cyphered += (str(int(PO[i]) ^ int(ICVData[i]))) # applying XOR among the data, WHERE ICVDATA = Data + ICV

  return cyphered

### Data decyphering
- XOR (⊕) between PO and the cyphered data received.

In [7]:
def decypher(PO, cyphered):
  decyphered = ''
  for i in range(0, len(cyphered)):
    decyphered += (str(int(PO[i]) ^ int(cyphered[i]))) # applying XOR
  return decyphered

### Matching results
Compares data values returning a boolean

In [55]:
def match(data1, data2):
  return data1 == data2

## Problem execution

### Input data
Defining the variables with the statement data, and then printing formatted on bits results.

In [37]:
data = 0x0123
key = 0x33
iv = 0x11
divCRC = '100000001' ## (x^8 + 1) CRC polynomial divisor
expected = 0x667E92

In [38]:
print("\033[1m{0}\033[0m {1} = {2}".format('Data:', hex(data), hexToBin(data, 8)))
print("\033[1m{0}\033[0m {1} = {2}".format('Key:', hex(key), hexToBin(key, 8)))
print("\033[1m{0}\033[0m {1} = {2}".format('IV:', hex(iv), hexToBin(iv, 8)))
print("\033[1m{0}\033[0m {1} = {2}".format('CRC divisor:', '(x^8 + 1)', divCRC))
print("\033[1m{0}\033[0m {1} = {2}".format('Expected:', hex(expected), hexToBin(expected)))

Data: 0x123 = 100100011
Key: 0x33 = 00110011
IV: 0x11 = 00010001
CRC divisor: (x^8 + 1) = 100000001
Expected: 0x667e92 = 011001100111111010010010


## Problem 1.1
Probe that the ciphered data for the clear text data 0x0123, key=0x33 and IV=0x11 is: **0x667E92**


In [50]:
# Creating a string with the binary value of the provided data, to pass this data to the crc() function
msg = hexToBin(data, 16)

# Converting the data to ICV (Integrity Check Value) using the crc() funcion. Being (x^8 + 1) the CRC polynomial.
ICV = crc(msg, divCRC)

# With the calculated ICV (using the CRC value), we generate the ICV+Data stream, that will be cyphered.
ICVData = msg + ICV

# Using the pnrg() function to generate PS and PO, by providing the IV (Initialization Vector) and the Key
PS, PO = pnrg(iv, key)

# Showing the obtained PO (16 bits from the data and 8 bits from the key)
for v in PO: print(v, end='')


011001110101110110110000

In [67]:
# Cyphering ICV + DATA with the obtained PO (XOR operation (⊕))
cyphered = cypher(PO, ICVData)

# Printing Cyphered Data and provided probe value (0x667E92) in hex and binary formats
print("\033[1m{0}\033[0m {1} = {2}".format('Cyphered:', hex(int(cyphered, 2)), cyphered))
print("\033[1m{0}\033[0m {1} = {2}".format('Expected:', hex(expected), hexToBin(expected)))

# Comparing if values are the same
print("\033[1m{0}\033[0m => {1}".format('Matching??', str(match(cyphered, hexToBin(expected)))))

Cyphered: 0x667e92 = 011001100111111010010010
Expected: 0x667e92 = 011001100111111010010010
Matching?? => True


In [68]:
# Decyphering the data - XOR operation (⊕) between PO and the cyphered Data.
# in this case we can use the same PO value since we are using the same key and IV values
decyphered = decypher(PO, cyphered)

# Comparing  hex value of input data and obtained decyphered value
print("\033[1m{0}\033[0m {1} = {2}".format('Dechypered:', hex(int(decyphered[:16],2)), decyphered))
print("\033[1m{0}\033[0m {1} = {2}".format('   ICVData:', hex(int(ICVData[:16],2)), ICVData))
print("\033[1m{0}\033[0m {1} = {2}".format('  Original:', hex(data), hexToBin(data, 16)))

# Comparing if values are the same
print("\033[1m{0}\033[0m => {1}".format('Matching??', str(match(decyphered, ICVData))))

Dechypered: 0x123 = 000000010010001100100010
   ICVData: 0x123 = 000000010010001100100010
  Original: 0x123 = 0000000100100011
Matching?? => True
